In [ ]:
!pip freeze
!pip3 install numpy
!pip3 install pandas
!pip3 install sklearn
!pip3 install matplotlib
!pip3 install pydotplus
!pip3 install six

Problem 2: Decisions, decisions

Implement ID3 from scratch

In [ ]:
%ls